In [ ]:
import json
import re

import sys
sys.path.append('lib')
from cleaning_helpers import remove_null_items_recursive

In [ ]:
raw_data = {}
# parse config file at "../config.json" for input_file_path
with open('../config.json') as config_file:
    config = json.load(config_file)
    input_file_path = ('../' + config.get('input_file_path')).replace('//', '/')

print("input_file_path: ", input_file_path)
with open(input_file_path, 'r') as f:
    raw_data = json.load(f)

raw_app_template = raw_data.get('page', {}) \
    .get('changesRecord', [])[0] \
    .get('payload', {}) \
    .get('migratedAppTemplate', {})

folders = raw_app_template.get('folders', [])
raw_data = raw_app_template.get('plugins', {})

## define string search functions for finding relations between components

In [ ]:

# search string for instances like `my_component2.trigger` etc...
g_trigger_regex = r"[a-zA-Z_0-9]+\.trigger"
def get_trigger_snippets(s: str):
    return [snippet.strip() for snippet in re.findall(g_trigger_regex, s)]

# my_component.trigger is a dependant; not a dependency
def filter_triggers_from_code_snippets(snippets: list[str]):
    # replace all instances of regex in string with empty string
    return [re.sub(g_trigger_regex, "", snippet) for snippet in snippets]

# search string for instances like {{component.data.etc...}}
def get_dependency_snippets(s: str):
    code_pattern = r"\{\{([^{}]*)\}\}"
    return filter_triggers_from_code_snippets(
        [snippet.strip() for snippet in re.findall(code_pattern, s)]
    )

#### test

In [ ]:
# test filter_triggers_from_code_snippets
test = ["test.trigger('test')", "test2.trigger('test_2')", "test3.trigger('test_3') test3.trigger('test_3') test4.trigger('test_4')"]
result = filter_triggers_from_code_snippets(test)
print(result)
assert result == ["('test')", "('test_2')", "('test_3') ('test_3') ('test_4')"]

# Parse Data

In [ ]:
parsed_data = {};
for key, _value in raw_data.items():
    value = remove_null_items_recursive(_value)
    dependency_snippets = get_dependency_snippets(json.dumps(value))
    trigger_snippets = get_trigger_snippets(json.dumps(value))

    parsed_data[key] = {
        'id': key,
        'type': value.get('type'),
        'subtype': value.get('subtype'),
        'parent': value.get('position2', {}).get('container') if value.get('position2') else None,
        'children': [id for id, v in raw_data.items() if v.get('position2') and v.get('position2', {}).get('container') == key],
        'folder': value.get('folder'),
        
        'value': value.get('template', {}).get('value'),
        'funcBody': value.get('template', {}).get('funcBody'),
        'query': value.get('template', {}).get('query'),
        'transformer': value.get('template', {}).get('transformer') if value.get('template', {}).get('enableTransformer', False) else None,
        'errorTransformer': value.get('template', {}).get('errorTransformer') if value.get('template', {}).get('enableErrorTransformer', False) else None,
        
        'runWhenPageLoads': value.get('template', {}).get('runWhenPageLoads'),
        'runWhenModelUpdates': value.get('template', {}).get('runWhenModelUpdates'),
        'events': value.get('template', {}).get('events', []),
        
        'dependency_snippets': dependency_snippets,
        'trigger_snippets': trigger_snippets,

        'dependencies': [key for key in raw_data.keys() if f"{key}." in " | ".join(dependency_snippets)],
        'triggers': [key for key in raw_data.keys() if key in [trigger.replace(".trigger", "") for trigger in trigger_snippets]]
    }

parsed_data = remove_null_items_recursive(parsed_data)

In [ ]:
with open('intermediate_outputs/parsed_data.json', 'w') as f:
    json.dump(parsed_data, f, indent=4)